1. Загружаем все необходимые библиотеки для работы

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import cv2
import glob
import tensorflow
import keras

from tensorflow.keras.utils import normalize
from sklearn.model_selection import train_test_split
import segmentation_models as sm

Segmentation Models: using `keras` framework.


2. Загружаем необходимые файлы для тренировки и для тестирования модели

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
#!unzip drive/My\ Drive/AITU/train.zip
#!unzip drive/My\ Drive/AITU/test.zip

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


3. Получаем с исходного файла все рисунки и маски для дальнейшего использования

In [3]:
SIZE = 256
image_dataset = []   
mask_dataset = []  

directory_path = glob.glob(r'dataset/imgs')[0]
for image_path in glob.glob(os.path.join(directory_path, "*.jpg")):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (SIZE, SIZE))
    image_dataset.append(image) 
image_dataset = np.array(image_dataset)


directory_path = glob.glob(r'dataset/masks')[0]
for image_path in glob.glob(os.path.join(directory_path, "*.png")):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE, SIZE))
    mask_dataset.append(image) 
mask_dataset = np.array(mask_dataset)

4. Нормализируем датасет с рисунками, а для масок делаем rescaling. Затем делим даннные на трэйн и тест сплиты


In [4]:
image_dataset = normalize(image_dataset)
mask_dataset = np.expand_dims(mask_dataset, axis=3)/255

X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size=0.2, random_state=0)

5. Настройка модели

In [5]:
sm.set_framework('tf.keras')
model = sm.Unet('resnet34',  encoder_weights='imagenet')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=5, 
                    validation_data=(X_test, y_test), 
                    shuffle=False)


Epoch 1/5
12/12 [==============================] - 292s 24s/step - loss: 0.6904 - accuracy: 0.6834 - val_loss: 0.3529 - val_accuracy: 0.9750
Epoch 2/5
12/12 [==============================] - 275s 23s/step - loss: 0.4055 - accuracy: 0.9724 - val_loss: 1.0640 - val_accuracy: 0.9761
Epoch 3/5
12/12 [==============================] - 283s 23s/step - loss: 0.2887 - accuracy: 0.9750 - val_loss: 1.8850 - val_accuracy: 0.9764
Epoch 4/5
12/12 [==============================] - 278s 23s/step - loss: 0.2139 - accuracy: 0.9759 - val_loss: 0.8883 - val_accuracy: 0.9764
Epoch 5/5
12/12 [==============================] - 277s 23s/step - loss: 0.1667 - accuracy: 0.9759 - val_loss: 0.2825 - val_accuracy: 0.9764


In [6]:
# Сохраняем полученную модель
model.save('segmentation_model')

INFO:tensorflow:Assets written to: segmentation_model/assets


6. Загружаем и открываем тестируемые данные

In [11]:
test_dataset = []  

test_path = glob.glob(r'imgs')[0]
for image_path in glob.glob(os.path.join(test_path, "*.jpg")):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (SIZE, SIZE))
    test_dataset.append(image) 

test_dataset = np.array(test_dataset)

7. Нормализуем для получения лучшего результата

In [12]:
test_dataset = np.expand_dims(normalize(test_dataset), 3)

8. Делаем предсказания для каждого рисунка в test_dataset

In [19]:
for i in range(len(test_dataset)):     
    test_img = cv2.resize(test_dataset[i], (256,256),cv2.IMREAD_COLOR)
    test_img = np.array(test_img, dtype=np.uint8)
    test_img = cv2.cvtColor(test_img, cv2.COLOR_RGB2BGR)
    test_img = np.expand_dims(test_img, axis=0)
    prediction = model.predict(test_img)
    prediction_image = prediction.reshape((256,256))
    image_name = 'Image0' + str(int(161) + i)+'.png'

In [20]:
plt.imsave(image_name, prediction_image, cmap='gray')

# Новый раздел

# Новый раздел

# Новый раздел